In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
sys.path.append(str(Path("../../src").resolve()))

In [2]:
from utils.parser import load_prompt, load_jsonl, save_jsonl

In [3]:
from utils.openai_lm import OpenAIModel

# model_name = "gpt-4-0314"
model_name = "gpt-35-turbo-0301"

model = OpenAIModel(model_name=model_name, cache_file=None, model_mode="chat_gpt", api_type="azure")
model.load_model()
output, response = model.generate([{"role": "user", "content": "Hello, how are you?"}])
output

'As an AI, I do not have feelings or emotions, but I am here to help you. How can I assist you today?'

In [4]:
prompt_path = "prompt.txt"
inst_path = f"../../scripts/step2_score_instruction/outputs/instructions_score.{model_name}.1000.txt"

prompt_template = load_prompt(prompt_path)
inst_list = load_jsonl(inst_path)
print(f"Number of instructions: {len(inst_list)}")

Number of instructions: 1000


In [12]:
import random
num_inst = len(inst_list)
# num_inst = 10
# random.seed(42)
# inst_list = random.sample(inst_list, num_inst)
print(f"Number of instructions: {len(inst_list)}")

Number of instructions: 10


In [13]:

for inst in inst_list:
    # print(para)
    context = inst["context"]
    instruction = inst["instruction"]
    prompt = prompt_template.substitute(instruction=instruction, context=context)
    inst["input_messages"] = [{"role": "user", "content": prompt}]
    # input_messages_list.append([{"role": "user", "content": prompt}])
    # task_list.append(task)
    # print(prompt)

In [14]:
from tqdm import tqdm
import re

output_path = f"outputs/instructions_score.{model_name}.{num_inst}.txt"
output_list = []
for i, inst in enumerate(tqdm(inst_list)):
    try:
        input_messages = inst["input_messages"]
        output, response = model._generate(input_messages, max_tokens=2048, temperature=0.0)
        # match Analysis: (xxx)\nScore: (xxx)
        output_list.append({"instruction_score": inst["score"], "answer": output, "instruction": inst["instruction"], "context": inst["context"]})
    except Exception as e:
        print(e)
    if i % 100 == 0:
        save_jsonl(output_list, output_path)
        print(f"Number of outputs: {len(output_list)}")
save_jsonl(output_list, output_path)
print(f"Number of outputs: {len(output_list)}")

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:05<00:51,  5.72s/it]

Number of outputs: 1


100%|██████████| 10/10 [01:37<00:00,  9.75s/it]

Number of outputs: 10
